#### Generamos una modificacion de one_hot_encoding para que no salte error de que no hay clase R, D, V

In [19]:
import torch
import random
import pandas as pd
from generate_data import Data
from evaluate import get_wrong_predictions_bycases, print_wrong_preds_bycases
from models import LSTM_Model
from sklearn.preprocessing import LabelEncoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paso 1: Inicializar generador de datos
data = Data(layer_size=15, interop_distances=[0], permutation_intervals=1)

# Paso 2: Generar muestras con f2='wu' y f3='rd'
def generate_atomicity_samples_rd(n=10):
    layer_size = 15
    base_f1 = ',.,_,,_,_,._.,__'
    base_f1 = base_f1[:layer_size].ljust(layer_size, '.')  # asegurar longitud
    samples = []

    for _ in range(n):
        f2 = ['.'] * layer_size
        pos_w = random.randint(0, layer_size - 5)
        pos_u = min(pos_w + 1 + random.randint(0, 3), layer_size - 1)
        f2[pos_w] = 'w'
        f2[pos_u] = 'u'
        f2 = ''.join(f2)

        f3 = ['_'] * layer_size
        pos_r = random.randint(0, layer_size - 5)
        pos_d = min(pos_r + 1 + random.randint(0, 3), layer_size - 1)
        f3[pos_r] = 'r'
        f3[pos_d] = 'd'
        f3 = ''.join(f3)

        samples.append([base_f1, f2, f3, 'A'])

    return samples

# Paso 3: Generar muestras artificiales
custom_samples = generate_atomicity_samples_rd(10)
separated = data.separate_string_chars(custom_samples)

# Paso 4: Forzar inclusión de todos los caracteres usados en el entrenamiento
caracteres_entrenamiento = list(".,_wurdc")  # Asegúrate de incluir todos los que se usaron
layer_size = data.layer_size
extra_samples = []

for c in caracteres_entrenamiento:
    f1 = c * layer_size
    f2 = c * layer_size
    f3 = c * layer_size
    extra_samples.append([f1, f2, f3, 'A'])

separated_extra = data.separate_string_chars(extra_samples)
separated += separated_extra  # Inyectamos muestras de relleno

# Paso 5: One-hot encoding
one_hotted = data.one_hot_encode(separated)

# Paso 6: Eliminar muestras de relleno
one_hotted = one_hotted[:-len(extra_samples)]

# Paso 7: Cargar modelo LSTM entrenado
experiment_name = "05per"
best_lstm_model = LSTM_Model(data, 16, 32, 8).to(device)
best_lstm_model.load_state_dict(
    torch.load('./bestmodels/best_LSTM_model_' + experiment_name, map_location=device)
)
best_lstm_model.eval()

# Paso 8: Preparar datos para evaluación
x_test = data.to_lstm_format(one_hotted[:, :-1])
y_test = torch.tensor([0] * len(custom_samples))  # 'A' = índice 0
original_inputs = [s[:3] for s in custom_samples]

# Paso 9: Evaluación
wrong_preds = get_wrong_predictions_bycases(best_lstm_model, x_test, y_test, original_inputs)
print_wrong_preds_bycases(wrong_preds, k=10)

# Paso 10: Guardar en CSV si hay errores
"""if wrong_preds:
    df = pd.DataFrame(wrong_preds, columns=['f1', 'f2', 'f3', 'expected', 'predicted'])
    df.to_csv('errores_violaciones_atomicidad.csv', index=False)
    print("🔁 Errores exportados a errores_violaciones_atomicidad.csv")
else:
    print("✅ El modelo clasificó correctamente todas las muestras.")
    """

c:\Users\franm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


10 Atomicity violation cases wrong predicted:
Sample [',.,_,,_,_,._.,_', '..........w..u.', '_r___d_________'] | Prediction V
Sample [',.,_,,_,_,._.,_', '.........wu....', '_________r___d_'] | Prediction V
Sample [',.,_,,_,_,._.,_', '...w.u.........', '_r_d___________'] | Prediction V
Sample [',.,_,,_,_,._.,_', '....w...u......', '____r__d_______'] | Prediction V
Sample [',.,_,,_,_,._.,_', '.......w...u...', 'rd_____________'] | Prediction V
Sample [',.,_,,_,_,._.,_', '......w...u....', '__________r___d'] | Prediction V
Sample [',.,_,,_,_,._.,_', '..wu...........', '______r__d_____'] | Prediction V
Sample [',.,_,,_,_,._.,_', '..........w...u', '______rd_______'] | Prediction V
Sample [',.,_,,_,_,._.,_', '.w...u.........', '__r__d_________'] | Prediction V
Sample [',.,_,,_,_,._.,_', '......wu.......', '________r___d__'] | Prediction V


0 DeadLock cases wrong predicted:


0 Data race cases wrong predicted:


0 Valid cases wrong predicted:




c:\Users\franm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
C:\Users\franm\AppData\Local\Temp\ipykernel_2560\3134426830.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case wher

'if wrong_preds:\n    df = pd.DataFrame(wrong_preds, columns=[\'f1\', \'f2\', \'f3\', \'expected\', \'predicted\'])\n    df.to_csv(\'errores_violaciones_atomicidad.csv\', index=False)\n    print("🔁 Errores exportados a errores_violaciones_atomicidad.csv")\nelse:\n    print("✅ El modelo clasificó correctamente todas las muestras.")\n    '